In [2]:
import polars as pl
import os

In [4]:
os.chdir('../../')
current_dir = os.getcwd()
print(current_dir)

dataset_path = os.path.join(current_dir, "data", "tennis_dataset_raw.csv")

d:\Cours_M2\webscraping\ml-webscrap-tennis


In [10]:
df = pl.read_csv(dataset_path)
df.shape

(6095, 57)

In [9]:
df.unique("url_match").shape

(3052, 57)

In [ ]:
import os
import polars as pl

current_dir = os.getcwd()
dataset_path = os.path.join(current_dir, "data", "tennis_dataset_raw.csv")
output_path = os.path.join(current_dir, "data", "tennis_dataset_symmetric.csv")

df = pl.read_csv(dataset_path)

def invert_match(row):
    inverted_row = row.copy()
    for col in row.keys():
        if col.startswith("player1_"):
            inverted_row[col.replace("player1_", "player2_")] = row[col]
        elif col.startswith("player2_"):
            inverted_row[col.replace("player2_", "player1_")] = row[col]
    inverted_row["target"] = 1 - row["target"]  
    return inverted_row

unique_matches = df.group_by("url_match").count().filter(pl.col("count") == 1)
unique_urls = unique_matches["url_match"].to_list()

inverted_matches = []
for row in df.filter(pl.col("url_match").is_in(unique_urls)).iter_rows(named=True):
    inverted_matches.append(invert_match(row))

# Ajouter les matchs inversés au DataFrame d'origine
df_inverted = pl.DataFrame(inverted_matches)
df_symmetric = pl.concat([df, df_inverted])

# Sauvegarder le DataFrame symétrisé
df_symmetric.write_csv(output_path)

print(f"Base de données symétrisée sauvegardée dans : {output_path}")


C:\Users\parha\AppData\Local\Temp\ipykernel_15636\4071500969.py:24: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  unique_matches = df.group_by("url_match").count().filter(pl.col("count") == 1)


Base de données symétrisée sauvegardée dans : d:\Cours_M2\webscraping\ml-webscrap-tennis\data\tennis_dataset_symmetric.csv
